In [2]:
import numpy as np
import time
import pandas as pd

from game_functions.environment import sbfeatures as sbf
from game_functions.model import brain

2017-06-17 20:51:12,435 INFO: Imported existing <module 'comtypes.gen' from 'C:\\Program Files\\Anaconda3\\lib\\site-packages\\comtypes\\gen\\__init__.py'>
2017-06-17 20:51:12,448 INFO: Using writeable comtypes cache directory: 'C:\Program Files\Anaconda3\lib\site-packages\comtypes\gen'


OSError: [WinError 193] %1 is not a valid Win32 application

In [ ]:
env = sbf.Environment()

In [ ]:
env.controller(initialize=True, close=True)
#env.controller(initialize=True, opens=True)
#env.action(5, delay=0.1)

In [4]:
env.make()

launching game...


In [6]:
env.start_game(botsOn=True, mode='easy')

In [8]:
env.initalize_memory()

hwnd: 3609934
pid: c_long(12680)


In [9]:
print(env.observation())
#env.__grab_addresses__()
#env.__grab_features__()

[100.0, 4.0, -7.165, 100.0, -4.6, -7.0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 4.0, -4.6, -7.165, -7.0]


In [10]:
print(env.p1_score, env.p2_score)

-25000 -25000


In [37]:
env.reward_structure(p1_death=-1, p2_death=1, p1_damage=-0.01, p2_damage=0.01)

In [38]:
#print(env.features)
print(env.observation())
print(env.reward(stale_observation=env.features, 
                 new_observation=env.observation()))
print(len(env.observation()))

[56.0, 12.165, -7.165, 100.0, -3.7, -3.8, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 14.165, 1.8999999999999995, 0.0030000000000001137, 0.40000000000000036]
0.0
22


In [39]:

env.action(5, delay=0.1)

5

In [9]:
full_frame_shape = 24
input_shape_a = full_frame_shape - 4 - 1 # - 1 for rewards column
input_shape = input_shape_a - 1

Qas = brain.keras_model(inputs=input_shape_a, dropouts=0.2, opti=1e-6, clipping=0.05, clipval=10)
Vs = brain.keras_model(inputs=input_shape, dropouts=0.2, opti=1e-6, clipping=0.05, clipval=10)

In [42]:
load_model = 1
if load_model == 1:
    Qas.load_weights("training\\model_weights\\16042017\\94_Qas_128ReD2_128ReD1_1_Adam1E6_MSE_L9_D10_EP15_BS150_Positive.keras")
    Vs.load_weights("training\\model_weights\\16042017\\94_Vs_128ReD2_128ReD1_1_Adam1E6_MSE_L9_D10_EP15_BS150_Positive.keras")

In [43]:
import time
import pandas as pd
import random

save_iter = 10000
backup_experience = 5000

train = 250

#decision making
entropy = 0.05
learning_rate = 0.9

env.observation()
episode = 0

# action settings
randomness = True
epsilon = 0
epsilon_base = 1
epsilon_decay = 1

#action_decay_rate = 0.11
action_decay_rate = 0.05
fd_decay = False
move_list = [-1 for i in range(8)]
last_action = 0
adva = pd.DataFrame()
moves_tracking = pd.Series([i for i in range(8)])

#score keeping
scores = []
p1_score = 0
p2_score = 0

#training
state_observation = []
state_action_observation = []

#setup
obs_old = env.features[:]
observations = []
rewards = []
positive_rewards = []
replay = []
reset_p2 = False
reset_p1 = False

while True:
    try:
        episode += 1

        if ((p1_score + p2_score) % 100) == 0:
            score = [episode, p1_score, p2_score]
            scores.append(scores)

        time.sleep(0.05)
        obs_new = env.observation()[:]
        time.sleep(0.05)
        observation_backup = obs_new[:]
        reward = env.reward(stale_observation=obs_old, new_observation=obs_new)
        obs_old = obs_new[:]

        if observation_backup[0] <= 0:
            if reset_p2:
                p2_score += 1
                reset_p2 = False
                print(p1_score/(p1_score+p2_score))
        else:
            reset_p2 = True
        if observation_backup[3] <= 0:
            if reset_p1:
                p1_score += 1
                reset_p1 = False
                print(p1_score/(p1_score+p2_score))
        else:
            reset_p1 = True

        #delete unwanted variables
        del obs_new[13] #cd2
        del obs_new[12] #cd1
        del obs_new[3] #p2h
        del obs_new[0] #p1h

        #----Qas
        #predictions = env.create_predictions(obs_new)
        #action_rewards = Qas.predict(predictions) # vector of estiamte reward

        #----Vs
        #vector of expected reward given state
        #expected_reward = Vs.predict(np.array(obs_new).reshape((1,18)))[0]

        #advantage = action_rewards - expected_reward
        #advantage = np.exp(learning_rate * (advantage - entropy))

        #advantage_before = np.array(advantage)
        
        #action decay
        #action_decay_dict = env.action_decay(last_action, 
        #                                 move_list, 
        #                                 decay_rate=action_decay_rate, 
        #                                 fire_decay=fd_decay)
        #for index, value in enumerate(advantage):
        #    advantage[index] = advantage[index] - action_decay_dict[index]
        #    if index == 4: #negatively enforce nothing
        #        advantage[index] = advantage[index] - 100
            
            
        #ad = pd.concat([moves_tracking, pd.DataFrame(advantage_before), pd.DataFrame(advantage)], axis=1)
        #adva = pd.concat([adva, ad])

        #action = np.where(advantage == np.amax(advantage))[0][0]
        #action = np.where(action_rewards == np.amax(action_rewards))[0][0] # greedy action
        
        #random action
        if randomness:
            epsilon = epsilon_base + 0.95 * np.exp(-episode/epsilon_decay)
            if random.random() < epsilon:
                action = random.randint(0, 7)
    
        action = env.action(action, delay=0.15)
        last_action = action
        #action = env.action_checker(action, reward, cooldownstates=obs_new[-2:])

        #csv backup
        observation_backup.append(action)
        observations.append(observation_backup)
        rewards.append(reward)
    
        #model
        state_observation.append(obs_new[:])
        obs_new.append(action)
        state_action_observation.append(obs_new[:])

        if reward < 0:
            reward = 0
        positive_rewards.append(reward)
        
        if reward != 0:
            print('Reward:', reward, 'p1', observation_backup[0], 'p2',observation_backup[3])

        if (episode % train) == 0:
            #Qas.fit(state_action_observation, positive_rewards, nb_epoch=50, batch_size = 10, verbose=0)
            #Vs.fit(state_observation, positive_rewards, nb_epoch=50, batch_size = 10, verbose=0)

            replay.append([observations, rewards])
            observations = []
            rewards = []

            state_observation = []
            state_action_observation = []
            positive_rewards = []

            print('Experience:', episode+1,' Randomness:', epsilon*100)

        if (episode % backup_experience) == 0:
            name = str(episode) + '_experiencebackup.csv'
            exp = pd.DataFrame(replay)
            exp.to_csv(name, index=False)
            print('saving experience...')
            replay = []
        if episode % save_iter == 0:
            Qas.save_weights("SB-Qas-v0.keras")
            Vs.save_weights("SB-Vs-v0.keras")
        if episode % 500:
            adva = pd.DataFrame()
    except Exception:
        episode -= 1
        print('Error Reading Memory')
        time.sleep(0.25)

Reward: 0.66 p1 56.0 p2 34.0
0.0
Experience: 251  Randomness: 100.0
0.0
0.3333333333333333
Reward: 1.0 p1 100.0 p2 -10.0
Reward: 0.66 p1 48.0 p2 34.0
0.25
0.4
Reward: 1.0 p1 100.0 p2 -10.0
Experience: 501  Randomness: 100.0
Reward: 0.22 p1 26.0 p2 78.0
0.3333333333333333
Reward: 0.22 p1 100.0 p2 56.0
0.2857142857142857
Experience: 751  Randomness: 100.0
Reward: 0.41000000000000003 p1 78.0 p2 15.0
0.375
Reward: 1.0 p1 78.0 p2 -7.0
Reward: 0.22 p1 78.0 p2 78.0
0.3333333333333333
0.3
Error Reading Memory
Error Reading Memory
Experience: 1001  Randomness: 100.0
0.2727272727272727
0.25
0.23076923076923078
Experience: 1251  Randomness: 100.0
Reward: 0.54 p1 26.0 p2 24.0
0.2857142857142857
Reward: 1.0 p1 26.0 p2 -20.0
0.26666666666666666
0.25
0.23529411764705882
Error Reading Memory
Experience: 1501  Randomness: 100.0
Reward: 0.88 p1 100.0 p2 12.0
0.2777777777777778
Reward: 1.0 p1 100.0 p2 -35.0
Reward: 0.54 p1 4.0 p2 46.0
Experience: 1751  Randomness: 100.0
0.2631578947368421
Reward: 0.38 p1

KeyboardInterrupt: 

In [44]:
p1_score

261

In [45]:
p2_score

584

In [51]:
p1_score/p2_score

0.4469178082191781

In [49]:
env.p1_score + 25000

239

In [50]:
(env.p1_score + 25000)/(env.p2_score + 25000)

0.38486312399355876

In [30]:
env.switch

True

In [49]:
x = 0
y = 1

if x >= 1 or y > 1:
    print('yes')

In [51]:
0x10FF17e74

4562452084

In [20]:
env.process.get_feature(0xe19794c)

float


0.0

In [10]:
env.__update_score__()
#hex(env.p1_score_addr)
env.p1_score

9

In [17]:
env.p1_health_addr
#env.p1_score_prev

236550476

In [8]:
import addresses as ad
from ctypes import c_long, c_float
env.process.get_feature(env.process.get_addr(ad.baseaddr['p1_health'], ad.offsets['p1_health'], handle=False))

49.0

In [11]:
env.process.get_addr(ad.baseaddr['p1_score'], ad.offsets['p1_score'], handle=True)


AttributeError: 'Environment' object has no attribute 'get_score'

In [ ]:
#rand 0.109 p1 = 18 p2 = 147 

In [20]:
p1_score/(p1_score+p2_score)

0.10909090909090909

In [782]:
#policy = 0.34057971014492755
p1_score/(p1_score + p2_score)

0.4711160964666293

In [783]:
rewards

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [727]:
adva.columns = ['move','before','after']
adva['before'] = adva['before']
adva['after'] = adva['after']

In [728]:
adva['diff'] = adva['before'] - adva['after']

In [729]:
adva

,move,before,after,diff
0,0,0.977789,0.927789,5.000001e-02
1,1,0.979904,0.979904,0.000000e+00
2,2,0.973201,0.973201,0.000000e+00
3,3,0.975463,0.975463,0.000000e+00
4,4,0.975488,0.975488,0.000000e+00
5,5,0.973870,0.973870,0.000000e+00
6,6,0.976309,0.976309,0.000000e+00
7,7,0.988005,0.988005,0.000000e+00
0,0,0.978525,0.976025,2.499998e-03
1,1,0.977846,0.977846,0.000000e+00


In [714]:
advantage

array([[-224.03890991],
       [   0.96109158],
       [   0.96138281],
       [   0.96445912],
       [   0.96721524],
       [   0.96994865],
       [   0.97100776],
       [ -14.02917957]], dtype=float32)

In [54]:
exp = pd.DataFrame(replay)
exp.to_csv('experience_1v.csv', index=False)
print('saving experienc_e...')

saving experienc_e...
